# Recursivity

You have already seen recursivity in CSE101. I am going to remind you what recursivity is, to give it a more formal treatment, to explain how it works in practice (with the notion of stack frame), and to introduce memoization --- an optimization technique that we are going to use later for dynamic programming.

## A first example

A function is said recursive when its body contains call to itself (either directly or indirectly). E.g., the following function is a (direct) recursive function that prints all the numbers from $n$ downto $0$:

In [1]:
def f(n):
    print(n)
    if n > 0:
        f(n-1)

f(5)

5
4
3
2
1
0


While the following ones are indirect recursive functions for computing (very inefficiently) if a number is odd or even:

In [2]:
def even(n):
    if n < 0:
        return even(-n)
    if n == 0:
        return True
    return odd(n-1)

def odd(n):
    if n < 0:
        return odd(-n)
    if n == 0:
        return False
    return even(n-1)

print(odd(-134))

False


Recursion is at the root of (theoretical) Computer Science and is tightly linked to the notion of indutive (or recursive) structure.

## Recursivity and induction

### Natural numbers 

We restrict ourself to direct induction here.

Recursivity and induction are closed notions. We start with the example of natural numbers, and its induction principle that allows to prove a property $P$ (over natural numbers) using the following principle:

$\left\{ \begin{array}{l}
    P(0) \\
    \forall n \in \mathbb{N} .\, P(n) \implies P(n+1)
\end{array} \right. \implies \forall n \in \mathbb{N} .\, P(n)$

In a similar way, we can define a function $f$ over natural numbers if we give its value $f(0)$ for $0$ and its value for $f(n+1)$ from $f(n)$. For example, the factorial function $n! = \prod_{1 \le i \le n} i$ could be recursively defined by induction as follow: $f(0) = 1$ and $f(n+1) = (n+1) \times f(n)$. In python, this gives the following code:

In [3]:
def factorial(n):
    if n == 0:
        return 1 # base case
    return n * factorial(n-1) # inductive case

for i in range(10):
    print("{}! = {}".format(i, factorial(i)))

0! = 1
1! = 1
2! = 2
3! = 6
4! = 24
5! = 120
6! = 720
7! = 5040
8! = 40320
9! = 362880


### Inductive structure

We can extend this to any data-type that comes equipped with a well-founded order. A pair $(E, \preceq)$ is a well-founded if $\preceq$ is a pre-order on $E$ (i.e. it is reflexive and transitive) and there is no $\prec$-infinite chains (where $x \prec y \iff (x \preceq y) \wedge x \ne y$), i.e. there is no $\prec$-decreasing families $\{ \alpha_i \}_{i \in \mathbb{N}}$ (i.e. such that $i < j \implies \alpha_i \succ \alpha_j$).

Well-founded structure comes with an induction principle. Assume that $P$ is a predicate over $E$. Then, we have that:

$$(\forall x \in E .\, (\forall y \in E .\, y \prec x \implies P(y)) \implies P(x))
    \implies \forall x \in E .\, P(x)$$

Natural numbers $(\mathbb{N}, \leq)$ equipped with their natural ordering is a well-founded structure. The induction principle associated to it is what you know as the **strong induction principle** of natural numbers.

A function $f$ is defined inductively over $(E, \preceq)$ if, for any $x \in E$, $f(x)$ is expressed for the $f(y)$'s s.t. $y \prec x$. This includes the factorial function ($f(n+1) = (n+1) \times f(n)$ and $n \prec n+1$), or the $even$ predicate ($even(0) = \mathop{\top}$, $even(1) = \mathop{\perp}$ and $even(n+2) = even(n)$ and $n \prec n+2$).

But we are not restricted to the case $E = \mathbb{N}$. For example, the Ackermann function:

$$ A(m, n) = \left\{
  \begin{aligned}
    n + 1 && \text{if $m = 0$} \\
    A(m-1, n) && \text{if $m > 0$ and $n = 0$} \\
    A(m-1, A(m, n-1)) && \text{if $m > 0$ and $n > 0$}
  \end{aligned}
\right.$$

is a function that is inductively defined over $\mathbb{N} \times \mathbb{N}$ that is well-ordered with the lexicographic extension of $\leq$ --- i.e. $(x, y) \preceq (x', y')$ if $x < x'$ or $x = x' \wedge y \leq y'$.


Note that, in Python, you can write function that makes recursive call on values that are not strictly smaller than the function argument. E.g.

In [4]:
def myfunction(n):
    return n + myfunction(n+1)

In that case, you may obtain a function that does not terminate, as this is the case in the previous example. Note that a function $f$ may not be inductively well-defined for a structure $(E, \preceq)$ and terminates. In that case, this means that there exists a well-founded pre-order $\sqsubseteq$ for $E$ s.t. $f$ is inductively defined over $(E, \sqsubseteq)$ --- in fact, the ordering relation $\sqsubseteq$ could be defined in terms of $f$.

What you have to retain is that: if $f$ is a function that is inductively defined over a well-founded structure $(E, \preceq)$, then $f$ terminates. As you will see in algorithmic, this gives you a powerful tool to prove properties of $f$.

Note that, in some case, it is not even known if a function as a proper inductive definition or not. For instance, it is now know if the following function always terminates:

In [5]:
def syracuse(n):
    print(n)
    if n <= 1:
        return
    if n % 2: # n is odd
        syracuse(3 * n + 1)
    else:
        syracuse(n // 2)
        
syracuse(30)

30
15
46
23
70
35
106
53
160
80
40
20
10
5
16
8
4
2
1


## Stack frame

It is perfectly legal to use local variables in a recursive function. For instance, we could do the following:

In [6]:
def factorial(n):
    if n == 0:
        return 1
    aout = n
    aout = factorial(n-1) * aout
    return aout

for i in range(10):
    print("{}! = {}".format(i, factorial(i)))

0! = 1
1! = 1
2! = 2
3! = 6
4! = 24
5! = 120
6! = 720
7! = 5040
8! = 40320
9! = 362880


What we see from that example is that the local variable `aout` is not shared among the different **concurrent** calls --- if so, the result would have been incorrect, as shown by the following example:

In [7]:
aout = 0

def factorial_invalid(n):
    global aout

    if n == 0:
        return 1
    aout = n
    aout = factorial_invalid(n-1) * aout
    return aout

for i in range(10):
    print("{}! = {}".format(i, factorial_invalid(i)))

0! = 1
1! = 1
2! = 1
3! = 1
4! = 1
5! = 1
6! = 1
7! = 1
8! = 1
9! = 1


This means that Python (but this is not restricted to Python!) must have a place, different for each concurrent call, where it can store the local variables. For that purpose, Python uses an **execution stack**.

The execution stack is a data structure that stores information about the active functions (i.e. the functions that are called) of a program. In the stack, different information may be stored: the arguments of the function, the return address (i.e. where to return when the function call is over), the return value and also the local variables.

The execution stack is organized as a stack of **frames**. Each frame represents an active call and contains several information, as described before. When calling a function, the Python interpreter creates a new frame and pushed different values (the arguments of the function, the return address). Then, this frames is used by the callee to store its local variables.

<img src="files/stack.png" style="width: 40%">

When the callee is done, the return address is used to give the control back to the caller and the frame as the top of the execution stack is removed. The caller can then recover its local variables using the new top-frame.

Since there is one frame per active call, we see that concurrent calls to the same function used different memory locations for storing its local variables.

The execution stack is a limited in size. Too much recursion may lead to stack exhaustion:

In [8]:
def nothing(n):
    nothing(n+1)
    
nothing(0)

RecursionError: maximum recursion depth exceeded

This problem can be solved using tail-call recursion. For example, we could implementation the factorial as follows:

In [9]:
def factorial_tc(n, acc = 1):
    if n == 0:
        return acc
    return factorial_tc(n-1, n * acc)

print(factorial_tc(10))

3628800


In the code above, we see that the recursive calls are final (i.e. that they are always executed last). In that case, the function does not need its local variables & arguments anymore and we could destruct its frame right away... or reuse it for the next call of `factorial_tc`. This optimization is call **tail-call optimization**.

If tail-call optimization is applied on `factorial_tc`, then the execution stack never increases in size: there is only a single frame that is shared by all the concurrent calls. Hence, the stack cannot be exhausted.

However, note that Python does not implement the tail-call optimization.

## Memoization

In some cases, a naive implementation of a recursive function $f$ may lead to the computation of $f$ As an example, we can take the Fibonacci sequence $\mathcal{F}$ that is inductively defined by: $\mathcal{F}(0) = 0$, $\mathcal{F}(1) = 1$ and $\mathcal{F}(n+2) = \mathcal{F}(n+1) + \mathcal{F}(n)$.

Such a definition leads to the following naive implementation:

In [10]:
def fibonacci(n):
    if n == 0:
        return 0
    if n == 1:
        return 1
    return fibonacci(n-1) + fibonacci(n-2)

def timed(f, n):
    import datetime as dt
    
    start = dt.datetime.now()
    aout  = f(n)
    stop  = dt.datetime.now()
    
    print(n, aout, stop - start)

for i in range(38):
    timed(fibonacci, i)

0 0 0:00:00.000001
1 1 0:00:00.000001
2 1 0:00:00.000001
3 2 0:00:00.000001
4 3 0:00:00.000001
5 5 0:00:00.000002
6 8 0:00:00.000003
7 13 0:00:00.000004
8 21 0:00:00.000006
9 34 0:00:00.000009
10 55 0:00:00.000013
11 89 0:00:00.000021
12 144 0:00:00.000034
13 233 0:00:00.000052
14 377 0:00:00.000083
15 610 0:00:00.000135
16 987 0:00:00.000219
17 1597 0:00:00.000357
18 2584 0:00:00.000572
19 4181 0:00:00.000926
20 6765 0:00:00.001536
21 10946 0:00:00.002472
22 17711 0:00:00.003920
23 28657 0:00:00.006393
24 46368 0:00:00.010304
25 75025 0:00:00.016677
26 121393 0:00:00.027008
27 196418 0:00:00.044002
28 317811 0:00:00.071238
29 514229 0:00:00.115185
30 832040 0:00:00.186354
31 1346269 0:00:00.301174
32 2178309 0:00:00.487888
33 3524578 0:00:00.787376
34 5702887 0:00:01.268106
35 9227465 0:00:02.052218
36 14930352 0:00:03.357483
37 24157817 0:00:05.438373


We can see that the execution time is increasing dramatically, even for small values, whereas, the following iterative counterpart does not have this problem:

In [11]:
def iterative_fibonacci(n):
    r0, r1 = 0, 1
    
    while n > 0:
        r0, r1, n = r1, r0+r1, n-1
    return r0

for i in range(38):
    timed(iterative_fibonacci, i)

0 0 0:00:00.000002
1 1 0:00:00.000001
2 1 0:00:00.000001
3 2 0:00:00
4 3 0:00:00.000001
5 5 0:00:00.000001
6 8 0:00:00
7 13 0:00:00.000001
8 21 0:00:00.000001
9 34 0:00:00
10 55 0:00:00
11 89 0:00:00.000001
12 144 0:00:00.000001
13 233 0:00:00.000001
14 377 0:00:00.000001
15 610 0:00:00.000001
16 987 0:00:00.000001
17 1597 0:00:00.000001
18 2584 0:00:00.000001
19 4181 0:00:00.000001
20 6765 0:00:00.000001
21 10946 0:00:00.000001
22 17711 0:00:00.000001
23 28657 0:00:00.000001
24 46368 0:00:00.000002
25 75025 0:00:00.000001
26 121393 0:00:00.000001
27 196418 0:00:00.000002
28 317811 0:00:00.000002
29 514229 0:00:00.000002
30 832040 0:00:00.000001
31 1346269 0:00:00.000001
32 2178309 0:00:00.000001
33 3524578 0:00:00.000001
34 5702887 0:00:00.000001
35 9227465 0:00:00.000001
36 14930352 0:00:00.000002
37 24157817 0:00:00.000002


Why such a difference? Is recursive implementations are doomed to inefficiency? In fact, if we look closely at our implementation, we can see that we keep recomputing `fibonacci(k)` again and again for some values `k`. For example, for `k = 35`, a call to `fibonacci(35)` leads to a call to `fibonacci(34)` and `fibonacci(33)`. But the call to `fibonacci(34)` again leads to a call of `fibonacci(33)` --- so we are going to compute `fibonacci(33)` twice! And things go worst for `32` that we are going to recompute 3 times... in the end, the recursive definition of Fibonacci keeps recomputing things that we already computed in the past.

To overcome this problem, you can cache these intermediate values into a dictionary s.t. when we do a call to `fibonacci(k)`, we first look into the dictionary if we already computed it `k` and return it if so. This gives the following implementation:

In [12]:
def fibonacci_cached(n, cache):
    # Check the cache first
    if n in cache:
        return cache[n]
    # Cache miss, compute it
    if n == 0:
        aout = 0
    elif n == 1:
        aout = 1
    else:
        aout = fibonacci_cached(n-1, cache) + fibonacci_cached(n-2, cache)
    # Store the result in cache and return it
    cache[n] = aout; return aout
    
def fibonacci(n):
    return fibonacci_cached(n, {})

for i in range(38):
    timed(fibonacci, i)

0 0 0:00:00.000001
1 1 0:00:00.000001
2 1 0:00:00.000001
3 2 0:00:00.000001
4 3 0:00:00.000001
5 5 0:00:00.000001
6 8 0:00:00.000002
7 13 0:00:00.000002
8 21 0:00:00.000002
9 34 0:00:00.000003
10 55 0:00:00.000003
11 89 0:00:00.000003
12 144 0:00:00.000004
13 233 0:00:00.000003
14 377 0:00:00.000003
15 610 0:00:00.000003
16 987 0:00:00.000004
17 1597 0:00:00.000004
18 2584 0:00:00.000004
19 4181 0:00:00.000004
20 6765 0:00:00.000004
21 10946 0:00:00.000005
22 17711 0:00:00.000005
23 28657 0:00:00.000005
24 46368 0:00:00.000006
25 75025 0:00:00.000006
26 121393 0:00:00.000006
27 196418 0:00:00.000006
28 317811 0:00:00.000007
29 514229 0:00:00.000007
30 832040 0:00:00.000007
31 1346269 0:00:00.000007
32 2178309 0:00:00.000008
33 3524578 0:00:00.000007
34 5702887 0:00:00.000007
35 9227465 0:00:00.000007
36 14930352 0:00:00.000007
37 24157817 0:00:00.000008


Here, we obtain values that are comparable to the iterative version. Note that when doing a memoization, you will be face with the pervasive trade-off of computer science: by caching values, you speed-up your program, but you will also increase your memory consumption.

Of course, for the Fibonacci sequence, the iterative version is certainly the way to go. But there are problems where recursion is the way to go and where memoization will be central --- this will be the subject of a subsequence course.

As a final note, we can modify our last implementation a bit s.t. we store the number of time the `fibonacci` function is called when deactivating the memoization:

In [13]:
def fibonacci_c(n, counts):
    counts[n] = counts.get(n, 0) + 1
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fibonacci_c(n-1, counts) + fibonacci_c(n-2, counts)
    
def fibonacci(n):
    counts = {}; aout = fibonacci_c(n, counts)
    return (counts, aout)

for i in range(38):
    timed(fibonacci, i)

0 ({0: 1}, 0) 0:00:00.000001
1 ({1: 1}, 1) 0:00:00.000001
2 ({2: 1, 1: 1, 0: 1}, 1) 0:00:00.000001
3 ({3: 1, 2: 1, 1: 2, 0: 1}, 2) 0:00:00.000002
4 ({4: 1, 3: 1, 2: 2, 1: 3, 0: 2}, 3) 0:00:00.000002
5 ({5: 1, 4: 1, 3: 2, 2: 3, 1: 5, 0: 3}, 5) 0:00:00.000003
6 ({6: 1, 5: 1, 4: 2, 3: 3, 2: 5, 1: 8, 0: 5}, 8) 0:00:00.000004
7 ({7: 1, 6: 1, 5: 2, 4: 3, 3: 5, 2: 8, 1: 13, 0: 8}, 13) 0:00:00.000006
8 ({8: 1, 7: 1, 6: 2, 5: 3, 4: 5, 3: 8, 2: 13, 1: 21, 0: 13}, 21) 0:00:00.000009
9 ({9: 1, 8: 1, 7: 2, 6: 3, 5: 5, 4: 8, 3: 13, 2: 21, 1: 34, 0: 21}, 34) 0:00:00.000014
10 ({10: 1, 9: 1, 8: 2, 7: 3, 6: 5, 5: 8, 4: 13, 3: 21, 2: 34, 1: 55, 0: 34}, 55) 0:00:00.000023
11 ({11: 1, 10: 1, 9: 2, 8: 3, 7: 5, 6: 8, 5: 13, 4: 21, 3: 34, 2: 55, 1: 89, 0: 55}, 89) 0:00:00.000040
12 ({12: 1, 11: 1, 10: 2, 9: 3, 8: 5, 7: 8, 6: 13, 5: 21, 4: 34, 3: 55, 2: 89, 1: 144, 0: 89}, 144) 0:00:00.000058
13 ({13: 1, 12: 1, 11: 2, 10: 3, 9: 5, 8: 8, 7: 13, 6: 21, 5: 34, 4: 55, 3: 89, 2: 144, 1: 233, 0: 144}, 233) 0:00:00.

# Generators

An **generator** is a kind of cursor that allows to iterate over a sequence of objects. The iterator makes it possible to navigate through each object in a sequence without worrying about the underlying structure.

Iterators are pervasive in Python. For instance, `range(n)` is an iterator that allow to iterate over `0`, `1`, ..., `n-1`. A list is an iterator that allows to iterate over its arguments. You iterate over iterators using the `for i in iterator` construction:

In [14]:
for i in range(10):
    print(i)

0
1
2
3
4
5
6
7
8
9


Why not using directly list? First, iterators bring a higher level of abstraction, i.e. they allow you to programmatically generate your sequence. Second, by its very own nature, iterators are inexpensive objects in terms of memory usage as you do not build the full sequence in memory --- you may write infinite iterators!

## Implementing an iterator

Iterators are simply objects that come with two special methods called `.__next__()`. This method is responsible for returning the next value in the iterator. Moreover, objects may defined a special method called `.__iter__()` to return the actual implementation of the iterator.

For example, we could reimplement a simplified version of the `range` iterator:

In [15]:
class myrange:
    def __init__(self, bound):
        self.__current = 0
        self.__bound   = bound
        
    def __iter__(self):
        # We are our own iterator
        return self
        
    def __next__(self):
        if self.__current >= self.__bound:
            # Indicates that we are done with the iteration
            raise StopIteration
        aout = self.__current
        self.__current += 1
        return aout
    
for i in myrange(10):
    print(i)

0
1
2
3
4
5
6
7
8
9


We can also manually iterate over an object using the `iter()` and `next()` builtin, where `iter()` returns an actual iterator and `next()` returns the next value in the iterator. We detect the end of the iteration when the iterator raises the error (or exception) `StopIteration`.

In [16]:
it = iter(myrange(10))
while True:
    try:
        print(next(it))
    except StopIteration:
        break

0
1
2
3
4
5
6
7
8
9


We could for instance define an iterator that iterates over the (infinite) sequence of Fibonacci numbers

In [17]:
class Fibonacci:
    def __init__(self):
        self.__r0, self.__r1 = 0, 1
        
    def __iter__(self):
        # We are our own iterator
        return self
        
    def __next__(self):
        aout = self.__r0
        self.__r0, self.__r1 = self.__r1, self.__r0 + self.__r1
        return aout

fibo = iter(Fibonacci())
for _ in range(20):
    print(next(fibo))

0
1
1
2
3
5
8
13
21
34
55
89
144
233
377
610
987
1597
2584
4181


## The `yield` builtin

Python provides an idiomatic way to implement iterators via the `yield` builtin. Basically, any python function that contains the `yield` keyword is turned into an iterator that is going to enumerate all the values that are passed by the different calls to `yield`:

In [18]:
def myrange(n):
    i = 0
    while i < n:
        yield i
        i += 1
        
for j in myrange(10):
    print(j)

0
1
2
3
4
5
6
7
8
9


It is important to understand that we here face a non-local control-flow. When entering the `for`-loop, Python starts the execution of `myrange`, up to the execution of `yield`. At that point, Python **suspends** the execution of `myrange` and give the control-back to the `for`-loop (assignment the `yield`-ed value to the loop index `i`). When the loop body has been executed, Python resumes the execution of the iterator... up to next execution of `yield`, where it gives back the control to the `for`-loop. When we exit the iterator function, the iteration is over.

Here is our Fibonacci iterator reimplemented in terms of `yield`:

In [19]:
def fibonacci():
    r0, r1 = 0, 1
    while True:
        yield r0
        r0, r1 = r1, r0+r1
        
fibo = iter(fibonacci())
for _ in range(20):
    print(next(fibo))

0
1
1
2
3
5
8
13
21
34
55
89
144
233
377
610
987
1597
2584
4181


Note that it is perfectly valid to use an iterator in the implementation of some other iterators. However, we have to be a bit careful here and to explicitly `yield` the value of the subiterators:

In [20]:
def myiter(n):
    for i in range(0, n):
        yield i
    for i in range(-n, 0):
        yield i
        
for j in myiter(10):
    print(j)

0
1
2
3
4
5
6
7
8
9
-10
-9
-8
-7
-6
-5
-4
-3
-2
-1


Note that you can also `yield` all the values of a subiterator using the `yield from` syntax, as examplified below:

In [21]:
def myiter2(n):
    yield from range(0, n)
    yield from range(-n, 0)
    
for j in myiter(10):
    print(j)

0
1
2
3
4
5
6
7
8
9
-10
-9
-8
-7
-6
-5
-4
-3
-2
-1
